In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
import threading
import time
import requests
import json
import pandas as pd
import cx_Oracle
from datetime import datetime

In [11]:
def new_goods_scraping_to_df():
    url = 'https://search.danawa.com/dsearch.php?query=%EC%9E%90%EC%A0%84%EA%B1%B0'
    
    browser = webdriver.Chrome()
    browser.implicitly_wait(time_to_wait=10)
    browser.get(url)
    
    WebDriverWait(browser,10).until(expected_conditions.visibility_of_element_located((By.CLASS_NAME ,"prod_info")))
    
    html = browser.page_source
    html_parser = BeautifulSoup(html,features="html.parser")
    
    ####### 이름
    bike_name = html_parser.select("a.click_log_product_standard_title_")
    name_list = []

    for i in bike_name: 
        name = i.get_text()
        name_list.append(name)
    
    ###### 하이퍼링크
    bike_href = html_parser.select("a.click_log_product_standard_title_")
    href_list = []

    for i in bike_href: 
        href = i.attrs['href']
        href_list.append(href)
        
    ###### 이미지
    bike_img = html_parser.select("div.thumb_image a img")
    img_list = []
    for i in bike_img:
        try:
            img_list.append(i.attrs["data-src"])
        except KeyError:
            img_list.append(i.attrs["src"])
            
    img_list2 = []
    for img in img_list:
        if img != '//img.danawa.com/new/noData/img/noImg_160.gif':
            img_list2.append(img)
            
    ##### 가격
    bike_price2 = html_parser.select("li.rank_one strong")
    price_list = []

    for price in bike_price2:
          price= price.get_text() 
          price_list.append(price)
        
    price_list2 = []

    for price in price_list:
        if price != '상품비교':
            price_list2.append(price)
            
    price_list3 = []
    for price in price_list2:
        if price[1] == ',' or price[3] == ',':
            price_list3.append(price)
            
    
    new_bike_df = pd.DataFrame({'상품이름' : name_list,
                            '상품이미지' : img_list2,
                            '상품링크' : href_list,
                            '상품가격' : price_list3
                           })
    return new_bike_df

In [12]:
df = new_goods_scraping_to_df()

In [13]:
df['상품가격']=df['상품가격'].str.replace(',','')
df['상품가격']=df['상품가격'].astype('float64')

In [14]:
df.head(2)

,상품이름,상품이미지,상품링크,상품가격
0,랑케레이시 PANDA 해외구매,//img.danawa.com/prod_img/500000/465/359/img/1...,https://prod.danawa.com/info/?pcode=18359465&k...,1310720.0
1,SUMAD 해외구매,//img.danawa.com/prod_img/500000/793/412/img/1...,https://prod.danawa.com/info/?pcode=18412793&k...,294220.0


In [15]:
df.columns

Index(['상품이름', '상품이미지', '상품링크', '상품가격'], dtype='object')

In [16]:
def preprocessed_df_to_oracle(df):

    con = cx_Oracle.connect('bike/12345@192.168.0.78:1521/xe')
    cur = con.cursor()
    sql_insert = '''
            MERGE INTO NEW_GOODS 
        USING dual
            ON (NAME = :상품이름)
        WHEN NOT MATCHED THEN
            INSERT 
            VALUES (:상품이름2,:상품이미지,:상품링크,:상품가격) 
        WHEN MATCHED THEN
            UPDATE SET
                PRICE=:상품가격2
            '''
    
    for i in range(len(df)):
        상품이름 = df.iloc[i]['상품이름']
        상품이름2=상품이름
        상품이미지 = df.iloc[i]['상품이미지']
        상품링크 =df.iloc[i]['상품링크']
        상품가격 = df.iloc[i]['상품가격']
        상품가격2=상품가격




        cur.execute(sql_insert,(상품이름,상품이름2,상품이미지,상품링크,상품가격,상품가격2))

    con.commit()
    cur.close()
    con.close()

In [17]:
preprocessed_df_to_oracle(df)

In [18]:
def new_goods_collector():
    df = new_goods_scraping_to_df()
    preprocessed_df_to_oracle(df)

In [19]:
def news_scheduler():
    print('다나와 자전거 데이터 수집기 스케줄러 동작.\n')
    while True:
        new_goods_collector()
        print("수집완료.")
        time.sleep(6000) # 60분 주기로 데이터 수집

def print_main_menu():
    print('\n1. 다나와 자전거 실시간 데이터 구축')
    print('2. 스케줄러 종료')
    print('* 엔터: 메뉴 업데이트\n')

In [ ]:
while True:
    print_main_menu()
    print('아래행에 메뉴입력: ')
    selection = input()
    
    if selection == '':
        continue
    else:
        menu_num = int(selection)
        
    if(menu_num == 1):
        t = threading.Thread(target=new_goods_collector, daemon=True)
        t.start()
    elif(menu_num == 2):
        break
    elif (menu_num == 0):
        continue


1. 다나와 자전거 실시간 데이터 구축
2. 스케줄러 종료
* 엔터: 메뉴 업데이트

아래행에 메뉴입력: 
